In [90]:
import matplotlib.pyplot as plt
import pandas as pd 
from os import listdir
import numpy as np
import h5py
import cv2

RAW_DATA_FOLDER = 'data/'

IMAGES_FOLDER = RAW_DATA_FOLDER + 'images/'
TSV_FILE = RAW_DATA_FOLDER + 'airsim_rec.txt'

In [91]:
tsv_data = pd.read_csv(TSV_FILE, sep='\t')
steering_data = tsv_data['Steering']

labels = []
for i in steering_data:
    if i == -0.5:
        labels.append([1, 0, 0])
    elif i == 0.5:
        labels.append([0, 0, 1])
    else:
        labels.append([0, 0, 0])

In [92]:
images = [np.array(cv2.imread(IMAGES_FOLDER + f, 0)) for f in listdir(IMAGES_FOLDER)]

In [93]:
"""
    Split data into training and testing portions
"""
assert len(images) == len(labels)

training_portion = 0.7

end_index = int(len(images)*training_portion)

labels_training = np.array(labels[:end_index])
images_training = np.array(images[:end_index])

labels_testing = np.array(labels[end_index:])
images_testing = np.array(images[end_index:])


In [94]:

with h5py.File('data.h5df') as f:
    f.clear()
    
    labels_training_dset = f.create_dataset('label_training', shape=labels_training.shape,
                                           dtype=labels_training.dtype)
    images_training_dset = f.create_dataset('images_training', shape=images_training.shape,
                                           dtype=images_training.dtype)
    
    labels_testing_dset = f.create_dataset('label_testing', shape=labels_testing.shape,
                                          dtype=labels_testing.dtype)
    images_testing_dset = f.create_dataset('images_testing', shape=images_testing.shape,
                                          dtype=images_testing.dtype)
    
    labels_training_dset[:] = labels_training
    images_training_dset[:] = images_training
    
    labels_testing_dset[:] = labels_testing
    images_testing_dset[:] = images_testing
    
    